# ห้องปฏิบัติการเสริม - ฟังก์ชันการกระตุ้น ReLU 

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()
try:
  %matplotlib widget
  print("widget is already installed")
except:
  print("widget is not been installed, install now..")
  !pip install ipympl

In [ ]:

import requests
from pathlib import Path

url = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/deeplearning.mplstyle'
url2 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_common_C5.py'
url3 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/autils.py'
url4 = 'https://raw.githubusercontent.com/Smith-WeStrideTH/Advance_Learning_Algorithm_Course/refs/heads/main/work/lab_utils_relu.py'

response = requests.get(url)
with open('deeplearning.mplstyle', 'wb') as f:
  f.write(response.content)

response = requests.get(url2)
with open('lab_utils_common.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url3)
with open('autils.py', 'wb') as f:
  f.write(response.content)

response = requests.get(url4)
with open('lab_utils_relu.py', 'wb') as f:
  f.write(response.content)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.activations import linear, relu, sigmoid
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from autils import plt_act_trio
from lab_utils_relu import *
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


<a name="2"></a>
## 2 - ReLU Activation
สัปดาห์นี้ มีการแนะนำการกระตุ้นแบบใหม่ ซึ่งเรียกว่า Rectified Linear Unit (ReLU)"
$$ a = max(0,z) \quad\quad\text {# ReLU function} $$

In [2]:
plt_act_trio()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<img align="right" src="./images/C2_W2_ReLu_v1.png"     style=" width:380px; padding: 10px 20px; " >
การแปลเป็นภาษาไทย
"ตัวอย่างจากการบรรยายทางด้านขวาแสดงการประยุกต์ใช้ ReLU ในตัวอย่างนี้ คุณลักษณะ 'ความตระหนัก(awareness)' ที่ได้มาไม่ใช่แบบไบนารี แต่มีช่วงค่าต่อเนื่อง Sigmoid เหมาะที่สุดสำหรับสถานการณ์แบบเปิด/ปิดหรือแบบไบนารี ReLU ให้ความสัมพันธ์เชิงเส้นต่อเนื่อง นอกจากนี้ยังมีช่วง 'ปิด' ที่ผลลัพธ์เป็นศูนย์ คุณลักษณะ 'ปิด' ทำให้ ReLU เป็นการกระตุ้นแบบไม่เชิงเส้น ทำไมจึงจำเป็นต้องใช้? มาตรวจสอบด้านล่างนี้ 

### ทำไมต้องใช้ฟังก์ชันการกระตุ้นแบบไม่เชิงเส้น?  
<img align="left" src="./images/C2_W2_ReLU_Graph.png"     style=" width:250px; padding: 10px 20px; " > ฟังก์ชันที่แสดงเป็นฟังก์ชันเชิงเส้นแบบทีละชิ้น (piecewise linear) ความชันคงที่ในช่วงเชิงเส้นและเปลี่ยนแปลงอย่างฉับพลันที่จุดเปลี่ยน ที่จุดเปลี่ยนจะมีการเพิ่มฟังก์ชันเชิงเส้นใหม่ ซึ่งเมื่อเพิ่มเข้ากับฟังก์ชันที่มีอยู่แล้ว จะทำให้เกิดความชันใหม่ ฟังก์ชันใหม่จะถูกเพิ่มที่จุดเปลี่ยน แต่จะไม่ส่งผลต่อผลลัพธ์ก่อนถึงจุดนั้น ฟังก์ชันการกระตุ้นแบบไม่เชิงเส้นเป็นผู้รับผิดชอบในการปิดการใช้งานอินพุตก่อนและบางครั้งหลังจากจุดเปลี่ยน แบบฝึกหัดต่อไปนี้จะให้ตัวอย่างที่จับต้องได้มากขึ้น

แบบฝึกหัดนี้จะใช้เครือข่ายด้านล่างในปัญหาการถดถอยที่คุณต้องสร้างแบบจำลองเป้าหมายเชิงเส้นแบบแบ่งส่วน:
<img align="center" src="./images/C2_W2_ReLU_Network.png"     style=" width:650px; padding: 10px 20px; ">  
เครือข่ายมี 3 หน่วยในเลเยอร์แรก แต่ละหน่วยจำเป็นต้องสร้างเป้าหมาย หน่วย 0 ถูกตั้งโปรแกรมไว้ล่วงหน้าและคงที่เพื่อจับคู่เซ็กเมนต์แรก คุณจะต้องปรับน้ำหนักและอคติในหน่วย 1 และ 2 เพื่อสร้างแบบจำลองเซ็กเมนต์ที่ 2 และ 3 หน่วยเอาต์พุตยังคงที่และเพียงแค่รวมผลลัพธ์ของเลเยอร์แรก

โดยใช้แถบเลื่อนด้านล่าง ปรับน้ำหนักและอคติเพื่อให้ตรงกับเป้าหมาย เคล็ดลับ: เริ่มต้นด้วย `w1` และ `b1` และปล่อยให้ `w2` และ `b2` ป็นศูนย์จนกว่าคุณจะจับคู่เซ็กเมนต์ที่ 2 การคลิกแทนการเลื่อนจะเร็วกว่า หากคุณมีปัญหา อย่ากังวล ข้อความด้านล่างจะอธิบายรายละเอียดเพิ่มเติม

In [3]:
_ = plt_relu_ex()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

เป้าหมายของการฝึกนี้คือเพื่อชื่นชมว่าพฤติกรรมที่ไม่เป็นเชิงเส้นของ ReLU ให้ความสามารถที่จำเป็นในการปิดฟังก์ชันจนกว่าจะต้องการ มาดูกันว่ามันทำงานอย่างไรในตัวอย่างนี้
<img align="right" src="./images/C2_W2_ReLU_Plot.png"     style=" width:600px; padding: 10px 20px; "> 
"พล็อตทางด้านขวาประกอบด้วยผลลัพธ์ของหน่วยในเลเยอร์แรก เริ่มจากด้านบน หน่วย 0 มีหน้าที่รับผิดชอบส่วนแรกที่ทำเครื่องหมายด้วย 1 ทั้งฟังก์ชันเชิงเส้น $z$ และฟังก์ชันที่ตามหลัง ReLU$a$  จะแสดง คุณสามารถเห็นได้ว่า ReLU ตัดฟังก์ชันหลังช่วงเวลา [0,1] นี่เป็นสิ่งสำคัญเนื่องจากช่วยป้องกันไม่ให้หน่วย 0 แทรกแซงส่วนต่อไป

หน่วย 1 มีหน้าที่รับผิดชอบส่วนที่ 2 ที่นี่ ReLU ทำให้หน่วยนี้เงียบจนกว่า x จะถึง 1 เนื่องจากหน่วยแรกไม่ได้มีส่วนร่วม ความชันสำหรับหน่วย 1, $w^{[1]}_1$, จึงเป็นเพียงความชันของเส้นเป้าหมาย ต้องปรับค่าไบแอสเพื่อให้ผลลัพธ์เป็นลบจนกว่า x จะถึง 1 โปรดสังเกตว่าการมีส่วนร่วมของหน่วย 1 ขยายไปถึงส่วนที่ 3 ด้วย

หน่วย 2 มีหน้าที่รับผิดชอบส่วนที่ 3 ReLU ทำให้ผลลัพธ์เป็นศูนย์อีกครั้งจนกว่า x จะถึงค่าที่ถูกต้อง ความชันของหน่วย $w^{[1]}_2$, ต้องตั้งค่าเพื่อให้ผลรวมของหน่วย 1 และ 2 มีความชันที่ต้องการ ค่าไบแอสจะถูกปรับอีกครั้งเพื่อให้ผลลัพธ์เป็นลบจนกว่า x จะถึง 2

คุณสมบัติ "ปิด" หรือ "ปิดใช้งาน" ของฟังก์ชันการกระตุ้น ReLU ช่วยให้โมเดลสามารถเย็บส่วนเชิงเส้นเข้าด้วยกันเพื่อสร้างฟังก์ชันที่ไม่เชิงเส้นที่ซับซ้อน


## ขอแสดงความยินดี!
คุณคุ้นเคยกับ ReLU และความสำคัญของพฤติกรรมแบบไม่เชิงเส้นมากขึ้นแล้ว